# Installations

Install Gradio, then clone the Tensorflow Text to Speech Repository from Github, cd into it, install the TensorflowTTS and Soundfiles.

In [ ]:
!pip install --quiet gradio
!git clone https://github.com/TensorSpeech/TensorFlowTTS.git
!cd TensorFlowTTS
!pip --quiet install /content/TensorFlowTTS/
!pip --quiet install git+https://github.com/repodiac/german_transliterate
!pip --quiet install SoundFile


# Import modules and packages

Import the model and the text processor from the installed packages.

In [ ]:
import numpy as np
import soundfile as sf
import yaml
import tensorflow as tf
from tensorflow_tts.inference import TFAutoModel
from tensorflow_tts.inference import AutoProcessor
import gradio as gr

# Download the model

In [3]:
# initialize fastspeech2 model.
fastspeech2 = TFAutoModel.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")
# initialize mb_melgan model
mb_melgan = TFAutoModel.from_pretrained("tensorspeech/tts-mb_melgan-ljspeech-en")
# inference
processor = AutoProcessor.from_pretrained("tensorspeech/tts-fastspeech2-ljspeech-en")

Downloading:   0%|          | 0.00/125M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.83k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.57k [00:00<?, ?B/s]

# Inference function
Create the inference function for Gradio app

In [4]:
def inference(text):
      input_ids = processor.text_to_sequence(text)
      # fastspeech inference

      mel_before, mel_after, duration_outputs, _, _ = fastspeech2.inference(
          input_ids=tf.expand_dims(tf.convert_to_tensor(input_ids, dtype=tf.int32), 0),
          speaker_ids=tf.convert_to_tensor([0], dtype=tf.int32),
          speed_ratios=tf.convert_to_tensor([1.0], dtype=tf.float32),
          f0_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
          energy_ratios =tf.convert_to_tensor([1.0], dtype=tf.float32),
      )

      # melgan inference
      audio_before = mb_melgan.inference(mel_before)[0, :, 0]
      audio_after = mb_melgan.inference(mel_after)[0, :, 0]

      # save to file
      sf.write('./audio_before.wav', audio_before, 22050, "PCM_16")
      sf.write('./audio_after.wav', audio_after, 22050, "PCM_16")
      return './audio_after.wav'


# Gradio Interface and the app

In [7]:
inputs = gr.inputs.Textbox(lines=5, label="Input Text")
outputs =  gr.outputs.Audio(type="file", label="Output Audio")
title = "Tensorflow TTS"
description = "Gradio demo for TensorFlowTTS: Real-Time State-of-the-art Speech Synthesis for Tensorflow 2. To use it, simply add your text, or click one of the examples to load them."


/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/outputs.py:81: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import 

In [8]:
gr.Interface(inference, inputs, outputs, title=title, description=description).launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://af07bb237d39b1c3.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7f38337b9c90>,
 'http://127.0.0.1:7861/',
 'https://af07bb237d39b1c3.gradio.app')

# Test the inference function

In [14]:
inference("My name is Abuzar and I am majoring in Software Engineering.")

'./audio_after.wav'

In [15]:
from IPython.display import Audio


sound_file = 'audio_after.wav'
Audio(sound_file, autoplay=True)